## 0. Imports

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import countergen

## 1. Choose your dataset

In [ ]:
ds = countergen.Dataset.from_default("doublebind-positive-1token")

## 2. Choose your augmenters

In [ ]:
# augmenters = [countergen.SimpleAugmenter.from_default("gender")]

## 3. Convert!

In [ ]:
# aug_ds = ds.augment(augmenters)
aug_ds = countergen.AugmentedDataset.from_default("doublebind-positive-paraphrased")

## 4. Choose models to evaluate

In [ ]:
from countergenedit.evaluation import get_generative_model_evaluator, pt_to_generative_model
from countergenedit.tools.utils import get_device
from transformers import GPT2LMHeadModel
import torch

model_names = ["distilgpt2"]
device = get_device()
model: torch.nn.Module = GPT2LMHeadModel.from_pretrained(model_names[0]).to(device)
model_ev = get_generative_model_evaluator(pt_to_generative_model(model), "probability")

## 4b. Choose your metrics

In [ ]:
aggregator = countergen.aggregators.PerformanceStatsPerCategory()

## 5. Evaluate!

In [ ]:
results = countergen.evaluate(aug_ds.samples, model_ev, aggregator)

## 6. The results

In [ ]:
aggregator.display({"distilgpt2":results})

## 7. Edit...

In [ ]:
from countergenedit.editing import ActivationsDataset, get_mlp_modules
layers_dict = get_mlp_modules(model, [2,3])
layers = list(layers_dict.values())
act_ds = ActivationsDataset.from_augmented_samples(aug_ds.samples, model, layers)

In [ ]:
from countergenedit.editing.direction_algos import inlp
dirs = inlp(act_ds, n_training_iters=10)

In [ ]:
from countergenedit.editing import edit_model, get_edit_configs
configs = get_edit_configs(layers_dict, dirs, has_leftover=False)
new_model = edit_model(model, configs=configs)

In [ ]:
new_model_ev = get_generative_model_evaluator(pt_to_generative_model(new_model), "probability")
new_results = countergen.evaluate(aug_ds.samples, new_model_ev, aggregator)
aggregator.display({"newdistilgpt2":new_results})

In [ ]:
from countergenedit.editing.direction_algos import rlace
dirs = rlace(act_ds, n_dim=2, out_iters=100, num_clfs_in_eval=1, evalaute_every=50)

In [ ]:
from countergenedit.editing import edit_model, get_edit_configs
configs = get_edit_configs(layers_dict, dirs, has_leftover=False)
new_model = edit_model(model, configs=configs)

In [ ]:
new_model_ev = get_generative_model_evaluator(pt_to_generative_model(new_model), "probability")
new_results = countergen.evaluate(aug_ds.samples, new_model_ev, aggregator)
aggregator.display({"newdistilgpt2":new_results})